In [1]:
# Initialize required random seeds before importing anything else.
import random

import numpy as np
from tensorflow import set_random_seed

np.random.seed(42)
set_random_seed(42)
random.seed(42)

/home/qkt/.local/share/virtualenvs/sgan-techniques-WxACZbr4/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%matplotlib inline

import json

from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

import sgantech as sgt

Using TensorFlow backend.


In [3]:
sns.set_style('whitegrid')
color_palette = sns.color_palette()

In [4]:
(x_train, y_train), (x_test, y_test) = sgt.utils.load_preprocessed_mnist()
X = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))

In [5]:
def train_test_gan(gan_factory, x_train, y_train, x_test, y_test, epochs=40000, batch_size=32, test_steps=500, num_labels=None, verbose=True):
    classes = np.unique(y)
    num_classes = len(classes)
    
    quarter_batch = batch_size // 4
    cw1 = {0: 1, 1: 1}
    cw2 = {i: 10 / quarter_batch for i in range(num_classes)}
    cw2[num_classes] = 1 / quarter_batch
    
    if num_labels is None:
        x_train_labeled = x_train
        y_train_labeled = y_train
    else:
        x_train_labeled, _, y_train_labeled, _ = train_test_split(x_train, y_train, train_size=num_labels, stratify=y_train)
    
    disc, gen, combined = gan_factory()
    
    histories = []
    for epoch in range(epochs):
        # Select a random quater batch of images for real/fake
        real_fake_idx = np.random.randint(0, x_train.shape[0], quarter_batch)
        real_fake_imgs = x_train[real_fake_idx]
        # Select a random quarter batch of labeled images
        labeled_idx = np.random.randint(0, x_train_labeled.shape[0], quarter_batch)
        labeled_imgs = x_train_labeled[labeled_idx]
        # Sample noise and generate a half batch of new images
        noise = np.random.normal(0, 1, (quarter_batch * 2, 100))
        gen_imgs = gen.predict(noise)

        valid = np.ones((quarter_batch, 1))
        fake = np.zeros((quarter_batch, 1))

        labels = to_categorical(y_train_labeled[labeled_idx], num_classes=num_classes+1)
        fake_labels = to_categorical(np.full((quarter_batch, 1), num_classes), num_classes=num_classes+1)

        # Train the discriminator
        d_loss_real = disc.train_on_batch([real_fake_imgs, labeled_imgs], [valid, labels], class_weight=[cw1, cw2])
        d_loss_fake = disc.train_on_batch([gen_imgs[:quarter_batch], gen_imgs[quarter_batch:]], [fake, fake_labels], class_weight=[cw1, cw2])
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0, 1, (batch_size, 100))
        validity = np.ones((batch_size, 1))

        # Train the generator
        g_loss = combined.train_on_batch(noise, validity, class_weight=[cw1, cw2])
        
        if epoch % test_steps == 0:
            test_loss = disc.evaluate(
                [x_test, x_test],
                [np.ones((len(x_test), 1)), to_categorical(y_test, num_classes=num_classes+1)]
            )
            print(test_loss)
            histories.append((d_loss, g_loss, test_loss))

        # Plot the progress
        if verbose:
            print("%d [D loss: %f, acc: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss))
    
    test_loss = disc.evaluate(
        [x_test, x_test],
        [np.ones((len(x_test), 1)), to_categorical(y_test, num_classes=num_classes+1)]
    )
    histories.append(test_loss)
    
    return histories

## MLP

In [6]:
def dense_gan_factory():
    dense_d = sgt.discriminators.dense((28, 28, 1), 10)
    dense_g = sgt.generators.dense((100,), (28, 28, 1))
    dense_d, dense_g, dense_gd = sgt.gan.SGANBuilder(dense_g, dense_d, (28, 28, 1), (100,)).build()
    return dense_d, dense_g, dense_gd

In [7]:
# Baseline to compare to Keras-GAN SGAN implementation
dense_histories = train_test_gan(dense_gan_factory, x_train, y_train, x_test, y_test, epochs=50000, verbose=False)

/home/qkt/.local/share/virtualenvs/sgan-techniques-WxACZbr4/lib/python3.5/site-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


10000/10000 [==============================] - 0s 49us/step
[2.2246240142822264, 0.0006687384232878685, 4.448579292297364, 1.0, 0.101]
10000/10000 [==============================] - 0s 46us/step
[0.2999799731850624, 0.05961843948364258, 0.5403415083408356, 0.9839, 0.8297]
10000/10000 [==============================] - 0s 44us/step
[0.2161311832770705, 0.07307303901836276, 0.3591893277183175, 0.9752, 0.8946]
10000/10000 [==============================] - 1s 56us/step
[0.3200689273267984, 0.24706843256652355, 0.3930694226115942, 0.8947, 0.8753]
10000/10000 [==============================] - 1s 56us/step
[0.2444122080899775, 0.03218081219019368, 0.4566436036318541, 0.9909, 0.868]
10000/10000 [==============================] - 1s 56us/step
[0.21635690970569849, 0.10674314368776977, 0.32597067624628545, 0.948, 0.8981]
10000/10000 [==============================] - 1s 58us/step
[0.2035786152422428, 0.12498221894800662, 0.2821750132501125, 0.9424, 0.9171]
10000/10000 [========================

10000/10000 [==============================] - 1s 56us/step
[0.31287341623306275, 0.41129388732910155, 0.21445294565558434, 0.8473, 0.9436]
10000/10000 [==============================] - 1s 55us/step
[0.38085583572387693, 0.5189483971118927, 0.2427632746785879, 0.6997, 0.9357]
10000/10000 [==============================] - 1s 56us/step
[0.35770034642219545, 0.5400064956665039, 0.17539419580288232, 0.7011, 0.956]
10000/10000 [==============================] - 1s 55us/step
[0.43414911193847655, 0.5865520609855652, 0.2817461644232273, 0.6645, 0.9053]
10000/10000 [==============================] - 1s 56us/step
[0.5087850812911987, 0.6412317261695862, 0.37633843283653257, 0.6172, 0.8719]
10000/10000 [==============================] - 1s 56us/step
[0.46013143553733826, 0.5857758693695069, 0.3344870013535023, 0.6602, 0.9008]
10000/10000 [==============================] - 1s 56us/step
[0.4222005461215973, 0.5848980908393859, 0.2595030020684004, 0.6487, 0.9323]
10000/10000 [====================

## CNN

In [8]:
def cnn_gan_factory():
    cnn_d = sgt.discriminators.cnn((28, 28, 1), 10)
    cnn_g = sgt.generators.cnn((100,), (7, 7))
    return sgt.gan.SGANBuilder(cnn_g, cnn_d, (28, 28, 1), (100,)).build()

In [9]:
cnn_histories = train_test_gan(cnn_gan_factory, x_train, y_train, x_test, y_test, epochs=50000, verbose=False)

/home/qkt/.local/share/virtualenvs/sgan-techniques-WxACZbr4/lib/python3.5/site-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


10000/10000 [==============================] - 3s 315us/step
[1.5266181573867799, 0.6581736038208008, 2.3950627113342287, 0.9993, 0.155]
10000/10000 [==============================] - 3s 291us/step
[0.8894659698486328, 0.46102712755203246, 1.3179048151016235, 0.9985, 0.8395]
10000/10000 [==============================] - 3s 294us/step
[0.8684154569625855, 0.2218155288219452, 1.515015379524231, 1.0, 0.5102]
10000/10000 [==============================] - 3s 295us/step
[0.8795646850585938, 0.11662167875766755, 1.6425076969146728, 1.0, 0.3823]
10000/10000 [==============================] - 3s 299us/step
[1.130880242729187, 0.16031430435180663, 2.1014461736679078, 1.0, 0.2078]
10000/10000 [==============================] - 3s 297us/step
[1.1170890327453613, 0.06855704011917114, 2.1656210342407225, 1.0, 0.2203]
10000/10000 [==============================] - 3s 296us/step
[0.9929895433425904, 0.08758923246860505, 1.8983898563385009, 1.0, 0.3092]
10000/10000 [==============================] - 

10000/10000 [==============================] - 3s 290us/step
[0.38382933056354523, 0.5197834910869599, 0.24787516993358732, 0.6871, 0.9434]
10000/10000 [==============================] - 3s 294us/step
[0.32149990377426146, 0.49456463785171506, 0.148435169737041, 0.8001, 0.9654]
10000/10000 [==============================] - 3s 295us/step
[0.3964980451107025, 0.6520879076004028, 0.14090818375991657, 0.6185, 0.9661]
10000/10000 [==============================] - 3s 290us/step
[0.33857399230003354, 0.3980466520547867, 0.27910133403390647, 0.8444, 0.9272]
10000/10000 [==============================] - 3s 290us/step
[0.3495643633365631, 0.36394208245277404, 0.3351866435021162, 0.8546, 0.9103]
10000/10000 [==============================] - 3s 290us/step
[0.49586045055389405, 0.6330274152755737, 0.3586934882432222, 0.6534, 0.9086]
10000/10000 [==============================] - 3s 291us/step
[0.3612495900154114, 0.4429230386734009, 0.27957614121586083, 0.8155, 0.925]
10000/10000 [=============

## Semi-Supervised

In [10]:
dense_ss_histories = {}
for num_examples in [100, 300, 500, 1000]:
    print(num_examples)
    dense_ss_histories[num_examples] = train_test_gan(
        dense_gan_factory, x_train, y_train, x_test, y_test, epochs=50000, num_labels=num_examples, verbose=False
    )

100


/home/qkt/.local/share/virtualenvs/sgan-techniques-WxACZbr4/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/qkt/.local/share/virtualenvs/sgan-techniques-WxACZbr4/lib/python3.5/site-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


10000/10000 [==============================] - 1s 70us/step
[2.515849605178833, 0.0017914900243282319, 5.029907718658447, 1.0, 0.1001]
10000/10000 [==============================] - 1s 61us/step
[0.7649905866146087, 0.04282216841429472, 1.4871590037822724, 0.9947, 0.6915]
10000/10000 [==============================] - 1s 61us/step
[1.4135153090953827, 0.3277073676288128, 2.4993232554912566, 0.8585, 0.6444]
10000/10000 [==============================] - 1s 61us/step
[1.0723413139104843, 0.21002634667642414, 1.934656285238266, 0.9077, 0.6786]
10000/10000 [==============================] - 1s 61us/step
[2.042448923873901, 0.6546838081359864, 3.4302140325546264, 0.6972, 0.5851]
10000/10000 [==============================] - 1s 61us/step
[1.8237023423194885, 0.2081240452557802, 3.439280640888214, 0.9179, 0.576]
10000/10000 [==============================] - 1s 62us/step
[1.5538107577323914, 0.09709620481431484, 3.010525311756134, 0.9676, 0.6079]
10000/10000 [==============================] 

[8.032024713134765, 0.6378113030433655, 15.426238107299804, 0.6123, 0.026]
10000/10000 [==============================] - 1s 62us/step
[8.066327494812011, 0.7186646424293518, 15.413990353393554, 0.5277, 0.028]
10000/10000 [==============================] - 1s 61us/step
[8.169986408996582, 0.6324118275642395, 15.70756099243164, 0.6387, 0.0152]
10000/10000 [==============================] - 1s 62us/step
[7.923958697509765, 0.4054349669456482, 15.442482421875, 0.8378, 0.025]
10000/10000 [==============================] - 1s 61us/step
[7.319776390075684, 0.530298872089386, 14.10925389099121, 0.7274, 0.0921]
10000/10000 [==============================] - 1s 61us/step
[7.745778936767578, 0.772349182510376, 14.719208685302734, 0.4728, 0.0591]
10000/10000 [==============================] - 1s 61us/step
[7.874770028686523, 0.7530525532722473, 14.996487506103515, 0.4882, 0.0486]
10000/10000 [==============================] - 1s 60us/step
[6.378655696868896, 0.7177016582489014, 12.039609730529785

10000/10000 [==============================] - 1s 63us/step
[1.9072146780014039, 0.37628837089538575, 3.43814098777771, 0.8367, 0.4804]
10000/10000 [==============================] - 1s 63us/step
[1.3250133082389832, 0.2906609682559967, 2.359365653514862, 0.9091, 0.6054]
10000/10000 [==============================] - 1s 65us/step
[2.200373101711273, 0.431927108669281, 3.9688191028594972, 0.8007, 0.466]
10000/10000 [==============================] - 1s 64us/step
[2.4359854539871217, 0.4676918351173401, 4.40427907409668, 0.7719, 0.4072]
10000/10000 [==============================] - 1s 63us/step
[2.563080751609802, 0.79978935546875, 4.326372143936157, 0.5425, 0.4337]
10000/10000 [==============================] - 1s 64us/step
[2.0118855949401855, 0.6919116821289063, 3.331859508228302, 0.5793, 0.5406]
10000/10000 [==============================] - 1s 64us/step
[2.9019658069610594, 0.7160561367034912, 5.087875482940674, 0.6137, 0.394]
10000/10000 [==============================] - 1s 63us/

10000/10000 [==============================] - 1s 65us/step
[7.240650826263428, 0.7093758821487427, 13.771925750732422, 0.547, 0.0655]
10000/10000 [==============================] - 1s 65us/step
[5.307984998321533, 0.5742014317512513, 10.041768562316895, 0.6926, 0.2428]
10000/10000 [==============================] - 1s 64us/step
[5.630534813690185, 0.7009467061996459, 10.560122927856446, 0.5723, 0.2374]
10000/10000 [==============================] - 1s 63us/step
[6.160115161895752, 0.810018799495697, 11.510211517333984, 0.4759, 0.1708]
10000/10000 [==============================] - 1s 63us/step
[5.263171966552735, 0.5965653539657593, 9.92977859802246, 0.6677, 0.2628]
10000/10000 [==============================] - 1s 63us/step
[6.432643675231934, 0.5829336260795593, 12.282353727722167, 0.6766, 0.1324]
10000/10000 [==============================] - 1s 63us/step
[5.925945153808594, 0.4206330905437469, 11.431257217407227, 0.8247, 0.1844]
10000/10000 [==============================] - 1s 63

10000/10000 [==============================] - 1s 65us/step
[1.7744902782440186, 0.5860330963134766, 2.9629474660873414, 0.6556, 0.5677]
10000/10000 [==============================] - 1s 65us/step
[2.112378572273254, 0.6739541276931763, 3.5508030086517333, 0.6167, 0.4608]
10000/10000 [==============================] - 1s 65us/step
[2.3005557231903078, 0.8658748126983643, 3.7352366451263426, 0.4802, 0.464]
10000/10000 [==============================] - 1s 65us/step
[2.092632611656189, 0.8534998009681701, 3.3317654156684875, 0.4689, 0.549]
10000/10000 [==============================] - 1s 65us/step
[2.1114181774139404, 0.5701677483558655, 3.652668605041504, 0.6839, 0.5097]
10000/10000 [==============================] - 1s 65us/step
[1.8324682443618774, 0.6496458885192871, 3.0152905961990357, 0.6227, 0.6138]
10000/10000 [==============================] - 1s 66us/step
[2.4891701164245608, 0.611570820236206, 4.366769414520264, 0.6202, 0.3988]
10000/10000 [==============================] - 1

10000/10000 [==============================] - 1s 66us/step
[0.9422626517295838, 1.164494453239441, 0.7200308458924294, 0.5268, 0.7919]
10000/10000 [==============================] - 1s 67us/step
[0.33395760291665794, 0.04760699761789292, 0.6203082084655762, 0.9829, 0.842]
10000/10000 [==============================] - 1s 65us/step
[0.3362341431826353, 0.07619663880616427, 0.5962716463536024, 0.9733, 0.8632]
10000/10000 [==============================] - 1s 64us/step
[0.42933369869589805, 0.219424117384851, 0.6392432789737359, 0.9054, 0.87]
10000/10000 [==============================] - 1s 67us/step
[0.48681901820898055, 0.40958174135684966, 0.564056295183301, 0.8178, 0.8717]
10000/10000 [==============================] - 1s 67us/step
[0.32408938060812653, 0.037077117112092675, 0.6111016445791349, 0.9918, 0.876]
10000/10000 [==============================] - 1s 67us/step
[0.5101758904516697, 0.3408222050189972, 0.6795295798503794, 0.8226, 0.8727]
10000/10000 [==========================

10000/10000 [==============================] - 1s 68us/step
[1.3959858779907226, 0.707103707408905, 2.084868050670624, 0.6009, 0.6329]
10000/10000 [==============================] - 1s 67us/step
[1.673159322166443, 0.7419014142036437, 2.6044172277450564, 0.5331, 0.5389]
10000/10000 [==============================] - 1s 64us/step
[1.3737207359313965, 0.7183171666145325, 2.029124291038513, 0.5364, 0.6325]
10000/10000 [==============================] - 1s 63us/step
[1.1016588409423829, 0.49313592228889463, 1.7101817632198333, 0.7282, 0.6627]
10000/10000 [==============================] - 1s 67us/step
[1.6238709049224853, 0.4878020308494568, 2.7599397815704347, 0.7637, 0.5474]
10000/10000 [==============================] - 1s 68us/step
[1.4178063041687012, 0.4519706088066101, 2.38364200630188, 0.794, 0.5678]
10000/10000 [==============================] - 1s 66us/step
[1.3490315809249878, 0.5769094190597535, 2.121153745365143, 0.6873, 0.6404]
10000/10000 [==============================] - 1

In [11]:
cnn_ss_histories = {}
for num_examples in [100, 300, 500, 1000]:
    print(num_examples)
    cnn_ss_histories[num_examples] = train_test_gan(
        cnn_gan_factory, x_train, y_train, x_test, y_test, epochs=50000, num_labels=num_examples, verbose=False
    )

100


/home/qkt/.local/share/virtualenvs/sgan-techniques-WxACZbr4/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/qkt/.local/share/virtualenvs/sgan-techniques-WxACZbr4/lib/python3.5/site-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


10000/10000 [==============================] - 4s 354us/step
[1.4985946659088134, 0.6073497560501099, 2.3898395767211915, 1.0, 0.0982]
10000/10000 [==============================] - 3s 299us/step
[1.2441082008361817, 0.5166885471343994, 1.9715278526306153, 1.0, 0.6793]
10000/10000 [==============================] - 3s 296us/step
[0.9565770183563232, 0.1810712418794632, 1.732082797241211, 1.0, 0.4182]
10000/10000 [==============================] - 3s 298us/step
[1.179902975654602, 0.24691066746711732, 2.112895280838013, 1.0, 0.2873]
10000/10000 [==============================] - 3s 296us/step
[1.2106107084274291, 0.1988023513555527, 2.222419067764282, 1.0, 0.3451]
10000/10000 [==============================] - 3s 296us/step
[1.4543880603790282, 0.35323369922637937, 2.555542420196533, 1.0, 0.2802]
10000/10000 [==============================] - 3s 297us/step
[2.3444155162811278, 0.06679221549034119, 4.62203882522583, 1.0, 0.1963]
10000/10000 [==============================] - 3s 310us/ste

[1.2688737897872924, 0.0638268291413784, 2.4739207523345947, 1.0, 0.5367]
10000/10000 [==============================] - 3s 297us/step
[0.8779583947658539, 0.09455883738994598, 1.6613579594612122, 1.0, 0.6331]
10000/10000 [==============================] - 3s 296us/step
[0.7702453996658325, 0.039910281187295914, 1.5005805151939393, 1.0, 0.6908]
10000/10000 [==============================] - 3s 297us/step
[0.6661845278263092, 0.12240806167125702, 1.2099609944820404, 1.0, 0.7264]
10000/10000 [==============================] - 3s 294us/step
[0.9853360743522644, 0.05637223070263862, 1.9142999151229858, 1.0, 0.6492]
10000/10000 [==============================] - 3s 299us/step
[0.7592845076084137, 0.06540474535226821, 1.453164266872406, 1.0, 0.7154]
10000/10000 [==============================] - 3s 295us/step
[1.6259967693328858, 0.0489689478635788, 3.203024576950073, 1.0, 0.4728]
10000/10000 [==============================] - 3s 296us/step
[1.0608111229896546, 0.06744588174819946, 2.0541763

10000/10000 [==============================] - 3s 294us/step
[1.2871027811050415, 0.001415241764485836, 2.572790320587158, 1.0, 0.2517]
10000/10000 [==============================] - 3s 298us/step
[1.8164931203842163, 0.0007495769061148167, 3.632236665344238, 1.0, 0.2431]
10000/10000 [==============================] - 3s 298us/step
[1.293234081840515, 0.002590798008069396, 2.5838773635864256, 1.0, 0.2274]
10000/10000 [==============================] - 3s 297us/step
[1.023189059638977, 0.0008712671702727676, 2.045506849479675, 1.0, 0.3461]
10000/10000 [==============================] - 3s 299us/step
[0.939436173248291, 0.0019733432162553073, 1.8768990087509154, 1.0, 0.4994]
10000/10000 [==============================] - 3s 299us/step
[0.6958663699150085, 0.0030996551595628263, 1.3886330846786499, 1.0, 0.5558]
10000/10000 [==============================] - 3s 298us/step
[1.072591451358795, 0.002138513045758009, 2.143044388961792, 1.0, 0.394]
10000/10000 [==============================] -

10000/10000 [==============================] - 3s 297us/step
[6.812694115447998, 0.1212199536204338, 13.50416828918457, 0.999, 0.0219]
10000/10000 [==============================] - 3s 299us/step
[6.503511608886718, 0.07559082233905792, 12.931432405090332, 0.9999, 0.0306]
10000/10000 [==============================] - 3s 295us/step
[4.783632330322265, 0.13518796906471253, 9.432076692199708, 0.9989, 0.1158]
10000/10000 [==============================] - 3s 298us/step
[4.363242832946777, 0.1153192017197609, 8.611166456604003, 0.9989, 0.133]
10000/10000 [==============================] - 3s 294us/step
[1.6785003946304322, 0.11429309030771255, 3.242707699584961, 0.9995, 0.5229]
10000/10000 [==============================] - 3s 296us/step
[4.674165459442139, 0.06323242827653885, 9.285098510742188, 1.0, 0.1582]
10000/10000 [==============================] - 3s 297us/step
[6.933655207824707, 0.07009594838619232, 13.797214468383789, 1.0, 0.0301]
10000/10000 [==============================] - 3

10000/10000 [==============================] - 3s 297us/step
[0.547460053062439, 0.09338602545261383, 1.001534078025818, 1.0, 0.7023]
10000/10000 [==============================] - 3s 300us/step
[0.6244592350959778, 0.1846057774066925, 1.0643126929283142, 1.0, 0.648]
10000/10000 [==============================] - 3s 299us/step
[0.4785594481945038, 0.043086139678955075, 0.9140327557563782, 1.0, 0.7146]
10000/10000 [==============================] - 3s 299us/step
[0.7185782565116883, 0.04012835041284561, 1.3970281642913818, 1.0, 0.6128]
10000/10000 [==============================] - 3s 298us/step
[0.42006851572990417, 0.08523921325206757, 0.7548978179454804, 1.0, 0.764]
10000/10000 [==============================] - 3s 301us/step
[0.4492713203430176, 0.04853493835926056, 0.8500077037334443, 1.0, 0.731]
10000/10000 [==============================] - 3s 299us/step
[0.6387534721374512, 0.10296841201782227, 1.1745385330200195, 1.0, 0.6613]
10000/10000 [==============================] - 3s 29

10000/10000 [==============================] - 3s 302us/step
1000
10000/10000 [==============================] - 4s 394us/step
[1.5140812139511108, 0.6372059030532837, 2.3909565280914307, 1.0, 0.1162]
10000/10000 [==============================] - 3s 302us/step
[1.0164955419540405, 0.5880085718154907, 1.4449825128555298, 0.9165, 0.8345]
10000/10000 [==============================] - 3s 301us/step
[0.8740831069946289, 0.375041801738739, 1.3731244142532348, 1.0, 0.652]
10000/10000 [==============================] - 3s 301us/step
[1.2146071392059326, 0.38408376750946044, 2.0451305084228517, 1.0, 0.2531]
10000/10000 [==============================] - 3s 304us/step
[0.9918223151206971, 0.24289398896694184, 1.7407506462097169, 1.0, 0.324]
10000/10000 [==============================] - 3s 303us/step
[1.0608794764518739, 0.2888744164943695, 1.8328845401763916, 1.0, 0.2664]
10000/10000 [==============================] - 3s 299us/step
[0.9910440117835998, 0.2640333550453186, 1.718054666519165, 1

10000/10000 [==============================] - 3s 300us/step
[0.23521900000572205, 0.21849003794193267, 0.25194796276101844, 0.9967, 0.9362]
10000/10000 [==============================] - 3s 303us/step
[0.274181675362587, 0.21726124222278595, 0.33110210888162256, 0.9954, 0.9158]
10000/10000 [==============================] - 3s 300us/step
[0.2416612304329872, 0.15555986285209655, 0.3277625973193906, 0.9997, 0.9251]
10000/10000 [==============================] - 3s 300us/step
[0.20258528941869736, 0.14788392207622528, 0.25728665644340215, 1.0, 0.9398]
10000/10000 [==============================] - 3s 303us/step
[0.2446005278944969, 0.19606945247650145, 0.2931316035006195, 0.999, 0.9335]
10000/10000 [==============================] - 3s 301us/step
[0.2941709589242935, 0.24582966387271882, 0.34251225348077713, 0.994, 0.9265]
10000/10000 [==============================] - 3s 301us/step
[0.21853250337839128, 0.18144098439216613, 0.25562402309179305, 1.0, 0.935]
10000/10000 [================

In [12]:
import pickle

with open('vanilla_gan_results.pkl', 'wb') as f:
    pickle.dump({
        'dense_histories': dense_histories,
        'cnn_histories': cnn_histories,
        'dense_ss_histories': dense_ss_histories,
        'cnn_ss_histories': cnn_ss_histories,
    }, f)